<a href="https://colab.research.google.com/github/krishnbera/CSE486-INCM/blob/master/INCM_ND_Ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Install required packages

%%capture
!pip3 install brian2
!pip3 install --upgrade neurodynex3

In [ ]:
# @title Import required packages, libraries

%%capture
%matplotlib inline
import brian2 as b2
import matplotlib.pyplot as plt
import numpy as np
from neurodynex3.leaky_integrate_and_fire import LIF
from neurodynex3.tools import input_factory, plot_tools

##### Demo run of LIF neuron

In [ ]:
LIF.getting_started()

##### Print default parameters of LIF neuron

In [ ]:
LIF.print_default_parameters()

# Excercise 1.1 Minimal Current
In the absence of an input current, a LIF neuron has a constant membrane voltage vm=v_rest. If an input current drives vm above the firing threshold, a spike is generated. Then, vm is reset to v_reset and the neuron ignores any input during the refractroy period.

### [1.1.1] Question: minimal current (calculation)

Question: Minimal Current (calculation)
-
The voltage equation is given as follows:
\begin{equation*}
u(t) = u_{rest} + R I_{0} ( 1 - e^{-t/\tau} )
\end{equation*}
Making $I_0$ as subject, we get
\begin{equation*}
I_{0} = \frac{u(t) - u_{rest}}{R ( 1 - e^{-t/\tau} )}
\end{equation*}

In [ ]:
I_min = ??
print("I_min is %s" % (I_min))

### [1.1.2] Question: minimal current (simulation)
Use the value **i_min** you've computed and verify your result: inject a step current of amplitude i_min for 100ms into the LIF neuron and plot the membrane voltage. Vm should approach the firing threshold but *not* fire.

In [ ]:
# create a step current with amplitude = I_min
step_current = input_factory.get_step_current(
    t_start=5, t_end=100, unit_time=b2.ms,
    amplitude=I_min)  # set I_min to your value

# run the LIF model.
# Note: As we do not specify any model parameters, the simulation runs with the default values
(state_monitor,spike_monitor) = LIF.simulate_LIF_neuron(input_current=step_current, simulation_time = 100 * b2.ms)

# plot I and vm
plot_tools.plot_voltage_and_current_traces(
state_monitor, step_current, title="min input", firing_threshold=LIF.FIRING_THRESHOLD)
print("nr of spikes: {}".format(spike_monitor.count[0]))  # should be 0

# Excercise 1.2 f-I Curve
For a constant input current I, a LIF neuron fires regularly with firing frequency f. If the current is to small (I < I_min) f is 0Hz; for larger I the rate increases. A neuron's firing-rate versus input-amplitude relationship is visualized in an "f-I curve".
Question: f-I Curve and Refractoryness

### [1.2.1] Question: f-I Curve and refractoryness

Question: f-I Curve and Refractoryness
-
We now study the f-I curve for a neuron with a refractory period of 3ms (see :func:`.LIF.simulate_LIF_neuron` to learn how to set a refractory period).
<ol>
<li>Sketch the f-I curve you expect to see</li>
<li>What is the maximum rate at which this neuron can fire?</li>
<li>Inject currents of different amplitudes (from 0nA to 100nA) into a LIF neuron. For each current, run the simulation for 500ms and determine the firing frequency in Hz. Then plot the f-I curve. Pay attention to the low input current.</li>
</ol>


In [ ]:
f, I = list(), list(range(0, 101))
for i in I:
    step_current = input_factory.get_step_current(t_start=0, t_end=500, unit_time=b2.ms,amplitude=i * 1e-3 * b2.uamp)
    (state_monitor,spike_monitor) = LIF.simulate_LIF_neuron(input_current=step_current, simulation_time = 500 * b2.ms, abs_refractory_period=3*b2.ms)
    f.append(spike_monitor.count[0] / 0.5)
plt.plot(I, f)
plt.title('f-I Curve')
plt.xlabel("Current")
plt.ylabel("Frequency")
plt.show()

# Excercise 1.3 "Experimentally" estimate the parameters of a LIF neuron

A LIF neuron is determined by the following parameters: Resting potential, Reset voltage, Firing threshold, Membrane resistance, Membrane time-scale, Absolute refractory period. By injecting a known test current into a LIF neuron (with unknown parameters), you can determine the neuron properties from the voltage response.

### [1.3.1] "Read" the LIF parameters out of the Vm plot

<ol>
<li>Get a random parameter set</li>
<li>Create an input current of your choice.</li>
<li>Simulate the LIF neuron using the random parameters and your test-current. Note that the simulation runs for a fixed duration of 50ms.</li>
<li>Plot the membrane voltage and estimate the parameters. You do not have to write code to analyse the voltage data in the StateMonitor. Simply estimate the values from the plot. For the Membrane resistance and the Membrane time-scale you might have to change your current.</li>
<li>Compare your estimates with the true values.</li>
</ol>

In [ ]:
# get a random parameter. provide a random seed to have a reproducible experiment
random_parameters = LIF.get_random_param_set(random_seed=432)

# define your test current
test_current = input_factory.get_step_current(t_start=5, t_end=45, unit_time=b2.ms, amplitude= 10 * b2.namp)

# probe the neuron. pass the test current AND the random params to the function
state_monitor, spike_monitor = LIF.simulate_random_neuron(test_current, random_parameters)

# plot
plot_tools.plot_voltage_and_current_traces(state_monitor, test_current, title="experiment")
plt.show()

# print the parameters to the console and compare with your estimates
print("LIF generated paratemers")
LIF.print_obfuscated_parameters(random_parameters)

We can determine $u_{rest}$, $V_{\theta}$, $u_{reset}$ and $t_{refractory}$ from the graph.
- $u_{rest}$ = -60 mV
- $V_{\theta}$ = -15 mv
- $u_{reset}$ = -70 mV 
- $t_{refractory}$ = 2 ms

To find the parameters of the model ($R$ and $\tau$) we need two equations. Thus we compare the curves for two values of the current.
- For $I_1$ = 14 nA the mebrane voltage increases from $u_{rest}$ to $V_{\theta}$ in 10ms.
- For $I_2$ = 8.6 nA, the membrane voltage increased from $u_{rest}$ to $V_{\theta}$ in 20ms.

Thus using the equation stated earlier, we have
$$
V_{\theta} = u_{rest} + RI_1 (1 - e ^ {\frac{-10}{\tau}}) = u_{rest} + RI_2 (1 - e ^ {\frac{-20}{\tau}})
$$

On solving, we get $ e ^ {\frac{-10}{\tau}}$ = 0.628 or 1.

Now we solve for $R$ and $\tau$.

# Excercise 1.4: Sinusoidal input current and subthreshold response

In the subthreshold regime (no spike), the LIF neuron is a linear system and the membrane voltage is a filtered version of the input current. In this exercise we study the properties of this linear system when it gets a sinusoidal stimulus.

### [1.4.1] Sinusoidal input current

Question
-
Create a sinusoidal input current (see example below) and inject it into the LIF neuron. Determine the phase and amplitude of the membrane voltage.
We have the differential equation
\begin{equation*}
\tau \frac{du}{dt} = -(u - u_{rest}) + R I(t)
\end{equation*}
where \begin{equation*}
I(t) = I sin(\omega t)
\end{equation*}

In [ ]:
# note the higher resolution when discretizing the sine wave: we specify unit_time=0.1 * b2.ms
sinusoidal_current = input_factory.get_sinusoidal_current(200, 1000, unit_time=0.1 * b2.ms, amplitude= 2.5 * b2.namp, frequency=250*b2.Hz, direct_current=0. * b2.namp)

# run the LIF model. By setting the firing threshold to to a high value, we make sure to stay in the linear (non spiking) regime.
(state_monitor, spike_monitor) = LIF.simulate_LIF_neuron(input_current=sinusoidal_current, simulation_time = 120 * b2.ms, firing_threshold=0*b2.mV)

# plot the membrane voltage
plot_tools.plot_voltage_and_current_traces(state_monitor, sinusoidal_current, title="Sinusoidal input current")
print("nr of spikes: {}".format(spike_monitor.count[0]))

### [1.4.2] Question
For input frequencies between 10Hz and 1 kHz, plot the resulting amplitude of subthreshold oscillations of the membrane potential vs. input frequency.

### [1.4.3] Question
For input frequencies between 10Hz and 1 kHz, plot the resulting phase shift of subthreshold oscillations of the membrane potential vs. input frequency.

### [1.4.4] Question
To what type of filter (High-Pass, Low-Pass) does this correspond to?